<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/GluonTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gluonts

In [ ]:
import pandas as pd
import numpy as np
from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer

# Create a synthetic dataset
dates = pd.date_range(start='2020-01-01', periods=200, freq='D')
data = np.sin(np.linspace(0, 50, 200)) + np.random.normal(scale=0.1, size=200)
df = pd.DataFrame({'date': dates, 'value': data})

# Prepare the dataset for GluonTS
training_data = ListDataset(
    [{"start": df['date'][0], "target": df['value']}],
    freq="D"
)

# Initialize the estimator
estimator = DeepAREstimator(freq="D", prediction_length=30, trainer=Trainer(epochs=10))

# Train the model
predictor = estimator.train(training_data)

# Prepare future dates for prediction
future_dates = pd.date_range(start=dates[-1] + pd.Timedelta(days=1), periods=30, freq='D')
future_data = ListDataset(
    [{"start": df['date'][0], "target": np.append(data, np.zeros(30))}],
    freq="D"
)

# Generate forecasts
forecast_it, ts_it = predictor.predict(future_data)
forecasts = list(forecast_it)
ts = list(ts_it)

# Plot the forecasts
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(dates, data, label="Observed")
plt.plot(future_dates, forecasts[0].mean, label="Forecast")
plt.fill_between(future_dates, forecasts[0].mean - forecasts[0].stddev, forecasts[0].mean + forecasts[0].stddev, alpha=0.5)
plt.legend()
plt.show()